In [1]:
import pandas as pd
from datetime import datetime
import tqdm

# Загрузим контактные данные Партнеров

In [2]:
data = pd.read_excel("Объединенные контакты партнеров.xlsx", index_col=0)

In [3]:
data[:2]

,АдресЭП,НомерТелефона,КодПартнер,НаименованиеПартнера
0,<пустая строка>,<Пустая строка>,000000207,Элиот ЗАО
1,<пустая строка>,<Пустая строка>,000000208,БАП ХРОМДЕТ-ЭКОЛОГИЯ ООО


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45954 entries, 0 to 45953
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   АдресЭП               45954 non-null  object
 1   НомерТелефона         45954 non-null  object
 2   КодПартнер            45954 non-null  object
 3   НаименованиеПартнера  45954 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


# Обработка звонков

## Загрузим информация о звонках

In [5]:
calls = pd.read_excel("Звонки.xlsx")
calls[:2]

,Номер,Дата,Src,Dst,Answer,Duration,Billsec,Disposition,Направление,Src_Ссылка,Dst_Ссылка
0,115621234131246,03.07.2019 10:10:14,73833495640,119,03.07.2019 10:10:21,70,63,ANSWERED,Входящий,<Неопределено>,Скакун Анастасия Александровна
1,115621235311267,03.07.2019 10:12:12,74212358892,119,03.07.2019 10:12:21,201,192,ANSWERED,Входящий,<Неопределено>,Скакун Анастасия Александровна


## Удаление секунд

In [6]:
# Секунды не нужны, избавимся от них
for number, series in calls.iterrows():
    calls.at[number, 'Дата'] = series['Дата'].split(' ')[0]
    calls.at[number, 'Answer'] = series['Answer'].split(' ')[0]
    
# Конвертация в datetime
calls['Дата'] = pd.to_datetime(calls['Дата'], dayfirst = True)
calls['Answer'] = pd.to_datetime(calls['Answer'], dayfirst = True, errors = 'ignore')

calls = calls.sort_values(by=['Дата']) #Сортировка по СсылкаЗаказ и дате

#сбросим имеющиеся индексы и воссоздадим по порядку заново
calls.reset_index(inplace=True)

del calls['index'] # удалим старые индексы полученые после сброса на пред шаге

In [7]:
calls[:2]

,Номер,Дата,Src,Dst,Answer,Duration,Billsec,Disposition,Направление,Src_Ссылка,Dst_Ссылка
0,1156170179870,2019-06-28,120,89237392578,<Пустая,21,0,NOANSWER,Исходящий,Анисимов Александр Владимирович,Коробков Евгений Валерьевич
1,11561707641143,2019-06-28,73833495709,1001,<Пустая,5,0,NOANSWER,Входящий,<Неопределено>,<Неопределено>


In [8]:
calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39264 entries, 0 to 39263
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Номер        39264 non-null  object        
 1   Дата         39264 non-null  datetime64[ns]
 2   Src          39264 non-null  object        
 3   Dst          39264 non-null  object        
 4   Answer       39264 non-null  object        
 5   Duration     39264 non-null  int64         
 6   Billsec      39264 non-null  int64         
 7   Disposition  39264 non-null  object        
 8   Направление  39264 non-null  object        
 9   Src_Ссылка   39264 non-null  object        
 10  Dst_Ссылка   39264 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(8)
memory usage: 3.3+ MB


# Обработка e-mail

## Download Emails

In [9]:
imeils = pd.read_excel("Почта.xlsx")
imeils[:2]

,Ссылка,Дата,Номер,Получено,Отправитель,ПолучательВходящего,ПолучательИсходящего,Тема,УчетнаяЗапись
0,заявка пожарка от 22.04.2019 (Входящее письмо),22.04.2019 13:11:12,00-00014287,22.04.2019 14:26:49,"ООО ""СИБАР"" <sibar22@mail.ru>",Евгений Коробков,<NULL>,заявка пожарка,Евгений Коробков
1,Заявка от 14.05.2019 (Входящее письмо),14.05.2019 9:55:21,00-00015392,23.05.2019 9:15:39,"ООО ""СИЦПБ"" <sicpb@yandex.ru>",Скакун Анастасия,<NULL>,Заявка,Скакун Анастасия


In [10]:
imeils.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235015 entries, 0 to 235014
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Ссылка                235015 non-null  object
 1   Дата                  235015 non-null  object
 2   Номер                 235015 non-null  object
 3   Получено              235015 non-null  object
 4   Отправитель           235015 non-null  object
 5   ПолучательВходящего   235015 non-null  object
 6   ПолучательИсходящего  235015 non-null  object
 7   Тема                  235015 non-null  object
 8   УчетнаяЗапись         235015 non-null  object
dtypes: object(9)
memory usage: 16.1+ MB


## Удаление секунд

In [11]:
# Секунды не нужны, избавимся от них
for number, series in imeils.iterrows():
    imeils.at[number, 'Дата'] = series['Дата'].split(' ')[0]
    imeils.at[number, 'Получено'] = series['Получено'].split(' ')[0]
    
# Конвертация в datetime
imeils['Дата'] = pd.to_datetime(imeils['Дата'], dayfirst = True)
imeils['Получено'] = pd.to_datetime(imeils['Получено'], dayfirst = True, errors = 'ignore')

imeils = imeils.sort_values(by=['Дата']) #Сортировка по СсылкаЗаказ и дате

#сбросим имеющиеся индексы и воссоздадим по порядку заново
imeils.reset_index(inplace=True)

del imeils['index'] # удалим старые индексы полученые после сброса на пред шаге

## Денормализация 'Отправитель'

In [12]:
time_df = pd.DataFrame(data=None, columns=imeils.columns)

for num, series in tqdm.tqdm(imeils.iterrows(), total = len(imeils)):
    # Для Входящей почты
    hhh = series['Отправитель'].split(' ')
    addres_list = []
    for elem in hhh:
        if elem.find('@') != -1:
            adress = elem
            if adress.find('<') != -1:
                adress = adress[adress.find('<') + 1:]
            if adress.find('>') != -1:
                adress = adress[:adress.find('<')]
            # Проверка на дубликат
            if adress not in addres_list:
                addres_list.append(adress.lower())

    for adr in addres_list:
        time_Series = series.copy()
        time_Series['Отправитель'] = adr
        t_df = pd.DataFrame(data = time_Series.values.reshape(1, 9),
                            columns = imeils.columns)
        time_df = pd.concat([time_df, t_df])

100%|██████████████████████████████████████████████████████████████████████████| 235015/235015 [54:52<00:00, 71.37it/s]


In [13]:
imeils = time_df.copy()

## Денормализация 'ПолучательИсходящего'

In [14]:
time_df = pd.DataFrame(data=None, columns=imeils.columns)

for num, series in tqdm.tqdm(imeils.iterrows(), total = len(imeils)):
    # Для Входящей почты
    if series['ПолучательИсходящего'] != '<NULL>':

        hhh = series['ПолучательИсходящего'].split(' ')
        addres_list = []
        for elem in hhh:
            if elem.find('@') != -1:
                adress = elem
                if adress.find('<') != -1:
                    adress = adress[adress.find('<') + 1:]
                if adress.find('>') != -1:
                    adress = adress[:adress.find('<')]
                # Проверка на дубликат
                if adress not in addres_list:
                    addres_list.append(adress.lower())                   
                    
        for adr in addres_list:
            time_Series = series.copy()
            time_Series['ПолучательИсходящего'] = adr
            t_df = pd.DataFrame(data = time_Series.values.reshape(1, 9),
                                columns = imeils.columns)
            time_df = pd.concat([time_df, t_df])
    else:
        time_Series = series.copy()
        t_df = pd.DataFrame(data = time_Series.values.reshape(1, 9),
                                columns = imeils.columns)
        time_df = pd.concat([time_df, t_df])

100%|██████████████████████████████████████████████████████████████████████████| 235073/235073 [57:35<00:00, 68.03it/s]


In [15]:
#сбросим имеющиеся индексы и воссоздадим по порядку заново
time_df.reset_index(inplace=True)
del time_df['index']  # удалим старые индексы полученые после сброса на пред шаге

# Загрузим данные о продажам

In [16]:
orders = pd.read_excel('3-Финальные данные от 2020-02-27.xlsx', index_col=0)

# Добавим кол-во касаний на заказ

In [17]:
orders['cals'] = 0
orders['emails'] = 0

## Обработка email

In [18]:
partner_cod = orders['PartnerKod'].unique()
for code in tqdm.tqdm(partner_cod):
    emails_array = data.loc[(data['КодПартнер'] == code) & (
        data['АдресЭП'] != '<пустая строка>')]['АдресЭП'].unique()

    one_partner = orders.loc[orders['PartnerKod'] == code]
    unic_data = one_partner['ZakazData'].unique()
    kol_date = len(unic_data)

    for number in range(kol_date):
        sum_emails = 0

        if number == 0:
            for elem in emails_array:
                email_partner = time_df.loc[time_df['Отправитель'] == elem]
                email_delta = email_partner.loc[
                    email_partner['Дата'] <= unic_data[number]]
                sum_emails += email_delta['Номер'].unique().shape[0]

                email_partner = time_df.loc[time_df['ПолучательИсходящего'] ==
                                            elem]
                email_delta = email_partner.loc[
                    email_partner['Дата'] <= unic_data[number]]
                sum_emails += email_delta['Номер'].unique().shape[0]

                index_order = one_partner.loc[
                    one_partner['ZakazData'] <= unic_data[number]].index

        else:
            for elem in emails_array:
                email_partner = time_df.loc[time_df['Отправитель'] == elem]
                email_delta = email_partner.loc[
                    (email_partner['Дата'] > unic_data[number - 1])
                    & (email_partner['Дата'] <= unic_data[number])]
                sum_emails += email_delta['Номер'].unique().shape[0]

                email_partner = time_df.loc[time_df['ПолучательИсходящего'] ==
                                            elem]
                email_delta = email_partner.loc[
                    (email_partner['Дата'] > unic_data[number - 1])
                    & (email_partner['Дата'] <= unic_data[number])]
                sum_emails += email_delta['Номер'].unique().shape[0]

                #Получим индексы
                index_order = one_partner.loc[
                    (one_partner['ZakazData'] > unic_data[number - 1])
                    & (one_partner['ZakazData'] <= unic_data[number])].index

        if sum_emails > 0:
            orders.at[index_order, 'emails'] = sum_emails

100%|████████████████████████████████████████████████████████████████████████████| 4127/4127 [1:02:27<00:00,  1.10it/s]


## Обработка звонков

In [19]:
partner_cod = orders['PartnerKod'].unique()

for code in tqdm.tqdm(partner_cod):
    phone_number_array = data.loc[(data['КодПартнер'] == code) & (data['НомерТелефона'] != '<пустая строка>')]['НомерТелефона'].unique()
    one_partner = orders.loc[orders['PartnerKod'] == code]
    unic_data = one_partner['ZakazData'].unique()
    kol_date = len(unic_data)

    for number in range(kol_date):
        sum_calls = 0
        calls_index = pd.Index([])

        if number-1 == 0:
            # Вычисляем кол-во звонков
            for phone_number in phone_number_array:
                partner_calls = calls.loc[calls['Src'] == phone_number]
                calls_delta = partner_calls.loc[partner_calls['Дата'] <= unic_data[number]]
                sum_calls += calls_delta['Номер'].unique().shape[0]
                
                index_order = one_partner.loc[one_partner['ZakazData'] <= unic_data[number]].index

        else:
            for phone_number in phone_number_array:
                partner_calls = calls.loc[calls['Src'] == phone_number]
                calls_delta = partner_calls.loc[(partner_calls['Дата'] > unic_data[number-1]) & (partner_calls['Дата'] <= unic_data[number])]
                sum_calls += calls_delta['Номер'].unique().shape[0]
                
               #Получим индексы
                index_order = one_partner.loc[(one_partner['ZakazData'] > unic_data[number-1]) & (one_partner['ZakazData'] <= unic_data[number])].index
        
        if sum_calls > 0: 
            orders.at[index_order, 'cals'] = sum_calls


100%|██████████████████████████████████████████████████████████████████████████████| 4127/4127 [10:07<00:00,  6.80it/s]


# Результат добавления данных

In [20]:
orders[:2]

,AktVipRabot,Data,SummaAktaVipRabot,Quantity,KodStrokiAkt,ZakazKlienta,NomenklaturaKod,Nomenklatura,ZakazData,PartnerKod,...,ProdanoShtuk,KodStrokiRealiz,LinkOrder,SummaZakazano,RastoynieOtNSK,otMoscow,ПродажаСостоялась,ItogoProcentSkidki,cals,emails
0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2014-06-10 12:00:00,000001587,...,NaN,NaN,Заказ клиента ЛБ00-000001 от 10.06.2014 12:00:00,3690.0,1,3400,Нет,NaN,0,0
1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-12 12:00:00,000000067,...,NaN,NaN,Заказ клиента ЛБ00-003550 от 12.05.2015 12:00:00,5985.0,1300,2300,Нет,NaN,0,0


In [21]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146016 entries, 0 to 146015
Data columns (total 44 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   AktVipRabot                  2375 non-null    object        
 1   Data                         2375 non-null    datetime64[ns]
 2   SummaAktaVipRabot            2375 non-null    float64       
 3   Quantity                     2375 non-null    float64       
 4   KodStrokiAkt                 2375 non-null    float64       
 5   ZakazKlienta                 2375 non-null    object        
 6   NomenklaturaKod              2375 non-null    object        
 7   Nomenklatura                 2375 non-null    object        
 8   ZakazData                    146016 non-null  datetime64[ns]
 9   PartnerKod                   146016 non-null  object        
 10  OsnovnoyManager              146016 non-null  object        
 11  BiznesRegion              

# Сохраним данные

In [22]:
with pd.ExcelWriter(f'Продажи с касаниями клиентов.xlsx') as writer:
    orders.reset_index(drop=True).to_excel(writer, sheet_name='Data')